In [1]:
import numpy as np
import pandas as pd
import glob
import os

import torch
import torch.utils.data as data_utils
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence


In [2]:
path = r'odi_csv/' # use your path
all_files = glob.glob(path + "/*.csv")
innings2 = []
target=[]
all_teams=[] # total=24
all_teams.extend(['unknown'])
for filename in all_files:
    df0 = pd.read_csv(filename,usecols=[0])
    skips = df0.loc[ 'info' , : ].shape[0]

    df_info = pd.read_csv(filename,nrows=skips,skiprows=1,header=None)
    df_info = df_info.drop(columns=0).set_index(df_info.columns[1])
    winteam=None
    loseteam=None
    if 'winner' in df_info.index:
        winteam = df_info.loc['winner',:].values[0]
        teams = df_info.loc['team',:].values
        loseteam = teams[teams!=winteam][0]
    else:
        winteam = 'unknown'
        loseteam= 'unknown'
    
    df = pd.read_csv(filename,skiprows=skips+1,header=None)
    teams = df[3].unique()
    all_teams.extend(teams.tolist())
    
    df2 = df[df.columns[[1,2,7,8]]].set_index(df.columns[1]).drop(index=1)
    if df2.shape[0]>0:
        df2['winteam']=winteam
        df2['loseteam']=loseteam
        innings2.append(df2)
        i2team = df[df.columns[[1,3]]].set_index(df.columns[1]).drop(index=1).values[0,0]
        if (i2team==winteam):
            target.append(1)
        else:
            target.append(0)

# len(innings2) = 1556

In [3]:
all_teams = np.unique(np.array(all_teams)).tolist()
ind = np.arange(1,len(all_teams)+1).tolist()

for i in range(len(innings2)):
    df2 = innings2[i]    
    innings2[i] = df2.replace(all_teams,ind)

In [4]:
df2

,2,7,8,winteam,loseteam
1,,,,,
2,0.1,0,0,Ireland,Zimbabwe
2,0.2,0,0,Ireland,Zimbabwe
2,0.3,0,0,Ireland,Zimbabwe
2,0.4,0,0,Ireland,Zimbabwe
2,0.5,0,0,Ireland,Zimbabwe
2,0.6,2,0,Ireland,Zimbabwe
2,1.1,0,0,Ireland,Zimbabwe
2,1.2,1,0,Ireland,Zimbabwe
2,1.3,0,0,Ireland,Zimbabwe


In [127]:
features=[]
# create targets first
for i in range(len(innings2)):
    features.append(torch.tensor(innings2[i].values))

targets = torch.tensor(target)
# targets = torch.tensor([target,1-np.array(target)]).transpose(0,1) # One Hot
targets.size()

torch.Size([1556])

In [128]:
features = pad_sequence(features,batch_first=True)

features.size()

torch.Size([1556, 321, 5])

In [130]:
from sklearn.model_selection import train_test_split
X=features.numpy()
y=targets.numpy()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [135]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories='auto') 
data1 = ohe.fit_transform(X[:,3]).toarray() 
data2 = ohe.fit_transform(X[:,4]).toarray() 

array([1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [8]:
y_train = torch.tensor(y_train)

X_train = torch.tensor(X_train)
X_test  = torch.tensor(X_test)
y_test  = torch.tensor(y_test)

X_train = pad_sequence(X_train,batch_first=True)
X_test = pad_sequence(X_test,batch_first=True)
print(X_train.size())
print(X_test.size())

torch.Size([1244, 321, 3])
torch.Size([312, 321, 3])


In [9]:
class matchRNN(nn.Module):
    def __init__(self,insize,hsize,outsize):
        super(matchRNN,self).__init__()
        
        self.insize=insize
        self.hsize=hsize
        self.outsize = outsize
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=insize, hidden_size=hsize)
        self.fc_out = nn.Linear(in_features=hsize, out_features=outsize)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self,feat):
        feat = torch.tensor(feat[np.newaxis,:],dtype=torch.float32)
        batch_size = 1
        
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hsize))
        cell_state = torch.zeros((batch_size, self.hsize))
#         outputs = torch.empty((batch_size, self.outsize))
        for t in range(feat.size(1)):

            # for the first time step the input is the feature vector
            if t == 0:
                hidden_state, cell_state = self.lstm_cell(feat[:,t,:].view(1,-1), (hidden_state, cell_state))
                
            # for the 2nd+ time step, using teacher forcer
            else:
                hidden_state, cell_state = self.lstm_cell(feat[:,t,:].view(1,-1), (hidden_state, cell_state))
            
        out = self.fc_out(hidden_state)
#         outputs[t,:] = out
    
        return out

In [ ]:
# training

insize=X_train.size(2)
hsize=64
outsize=2    #binary classification
model = matchRNN(insize,hsize,outsize)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(60):
    epoch_loss=0
    for i in range(X_train.size(0)):
        model.zero_grad()
        op = model(X_train[i])
        loss = loss_function(op, y_train[i].view(-1))
        loss.backward()
        epoch_loss+=loss.data.item()
        optimizer.step()
    print(epoch, '  ',epoch_loss)

torch.save(model.state_dict(), './models/cric_prediction_test_train_split.pth')


/home/rahulchakwate/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [ ]:
# evaluation

model.eval()
corr=0
for i in range(X_train.size(0)):
    model.zero_grad()
    op = model(X_train[i])
    maxval,maxidx = torch.max(op,1)
    corr+= (maxidx==y_train[i]).numpy()[0]
train_acc = corr / y_train.size(0)
print('Train Acc = ',train_acc)

corr_test=0
for i in range(X_test.size(0)):
    model.zero_grad()
    op = model(X_test[i])
    maxval,maxidx = torch.max(op,1)
    corr_test+= (maxidx==y_test[i]).numpy()[0]
test_acc = corr_test / y_test.size(0)
print('Test Acc = ',test_acc)

In [ ]:
# model.load_state_dict(torch.load('./models/cric_prediction.pth'))

In [ ]:
# # test train split
# np.random.seed(0)
# ind = np.arange(targets.size(0))
# np.random.shuffle(ind)
# features = features[ind]
# target = targets[ind]
# split=1300
# feat_train = features[:split]
# feat_test  = features[split:]
# targ_train = targets[:split]
# targ_test = targets[split:]